In [1]:
# IMPORTS
import mysql.connector 
from database.connect import getConnection
import config.env

import matplotlib.pyplot as plt
import numpy as np
from notebooks.response2 import user_id_to_user, replied_sentiment, mentions_length, length


# Packages
import pandas as pd
import seaborn as sns
import matplotlib as plt
sns.set()

In [2]:
# Get connection
try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

In [3]:
#create a dictionary of all airline user_ids for later on
airlines= {56377143 : 'KLM', 106062176:'AirFrance',18332190:"British_Airways", 22536055:"AmericanAir",
           124476322:"Lufthansa",26223583:'AirBerlin',2182373406:'AirBerlin assist',38676903:"easyJet",1542862735:"RyanAir",
           253340062:"SingaporeAir",218730857:"Qantas",45621423:"EtihadAirways",20626359:"VirginAtlantic"}

In [4]:
# query relevant data and fetch it
cursor = connection.cursor()
query_columns = ['id', 'text', 'in_reply_to_status_id', 'timestamp_ms', 'user_id', 'language', 'mentions', 'airlines',
                 'sentiment', 'reply_time']
query="""SELECT id, text, in_reply_to_status_id, user_id, language, mentions, airlines, sentiment, reply_time
        FROM tweets"""
cursor.execute(query)
tweets = cursor.fetchall()

In [ ]:
# Creating the dataframe
df_columns = ['response_time', 'self_sentiment', 'replied_sentiment', 'user_name', 'language',
              'mention_count', 'airline', 'tweet_length']
df_data = pd.DataFrame(columns=df_columns)

# Replace iteration with to dataframe.

for tweet in tweets:
    dic = {'response_time': 'Null', 'self_sentiment': 'Null', 'replied_sentiment': 'Null', 'user_name': 'Null', 'language': 'Null', 'mention_count': 'Null', 'airline': 'Null', 'tweet_length': 'Null'}
    tweet_id = tweet[0]
    tweet_text = tweet[1]
    in_reply_to = tweet[2]
    tweet_user = tweet[3]
    tweet_lan = tweet[4]
    tweet_mentions = tweet[5]
    tweet_airline = tweet[6]
    tweet_self_sentiment = tweet[7]
    response_time = tweet[8]
    # Response time -> int, unit: ms
    try:
        dic['response_time'] = response_time
    except Exception:
        pass   

    # Sentiment detection -> float, in range(-1, 1)
    try:
        dic['self_sentiment'] = tweet_self_sentiment
    except Exception:
        pass
    
    # Sentiment of parent tweet -> float, in range(-1, 1)
    try:
        dic['replied_sentiment'] = replied_sentiment(in_reply_to, cursor)
    except Exception:
        pass

    # Sentiment of parent tweet -> float, in range(-1, 1)
    try:
        if tweet_user in airlines:
            dic['user_name'] = airlines[tweet_user]
        else:
            dic['user_name'] = 'not an airline'
    except Exception:
        pass

    # Sentiment detection -> float, in range(-1, 1)
    try:
        dic['language'] = tweet_lan
    except Exception:
        pass

    # User mentions -> list(str, str, ...)
    try:
        dic['mention_count'] = mentions_length(tweet_mentions)
    except Exception:
        pass
    
     # Airline -> list(str)
    try:
        dic['airline'] = tweet_airline
    except Exception:
        pass   

    # Tweet length -> int
    try:    
        dic['tweet_length'] = length(tweet_text)
    except Exception:
        pass
    df_dict = pd.DataFrame([dic])
    df_data = pd.concat([df_data, df_dict], ignore_index=True)
    
df_data.count()

In [8]:
df_data.head()

,response_time,self_sentiment,replied_sentiment,user_name,language,mention_count,airline,tweet_length
0,0,-0.462126,Null,not an airline,en,3,[],140
1,0,0.164795,Null,not an airline,en,1,[british_airways],16
2,0,0.929742,Null,not an airline,en,3,[americanair],140
3,0,0.043662,Null,not an airline,en,3,[klm],140
4,0,-0.630411,Null,not an airline,en,0,[easyjet],77
